In [19]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
dicio={
'nan':0,'Mr.':1 ,'Mrs.':2,'Miss.':3 ,'Master.':4 ,'Planke':5,'Don.':6 ,'Rev.':7,'Ms.':8 ,'Major.':9 ,'Col.':10 ,'Capt.':11 }

In [21]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
dataframe =[train_df,test_df]
for i in dataframe:
    i['Sex'] = i['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
    i['Embarked'] = i['Embarked'].fillna('S')
    i['Embarked'] = i['Embarked'].map( {'S': 1, 'C': 0,'Q': 2} ).astype(int)    
    i['Cabin']=i['Cabin'].fillna('0')
    i['Fare']=i['Fare'].fillna('0')
    #i['Age']=i['Age'].fillna('0')
    i['NameSr']=i.Name.str.split().str[1].map(dicio).astype(str)#.loc[i.NameSr == 'nan','NameSr']=0
    i.loc[i.NameSr == 'nan','NameSr']=0
    i.loc[i.Cabin != '0','Cabin']=1
    #i['Age_bin'] = pd.cut(i['Age'], bins=[0,12,20,40,120], labels=['Children','Teenage','Adult','Elder'])
    #i.drop(['PassengerId','Ticket','Name'],axis = 1, inplace=True)
    

In [22]:
test_df['Age'].fillna(test_df['Age'].median(), inplace = True)

In [23]:
for dataset in dataframe:
    dataset['Age_bin'] = pd.cut(dataset['Age'], bins=[0,12,20,40,120], labels=[1,2,3,4])

In [24]:
#train_df['NameSr']=train_df.Name.str.split().str[1].map(dicio).astype(str).loc[train_df.NameSr == 'nan','NameSr']=0
#train_df.loc[train_df.NameSr == 'nan','NameSr']=0
train_df.Age_bin.value_counts()

3    385
4    150
2    110
1     69
Name: Age_bin, dtype: int64

In [25]:
#train_df.Cabin.value_counts()
#train_df.Fare.isnull().sum()

In [26]:
#train_df.Cabin.value_counts()
Y_train=train_df['Survived']
X_train=train_df[['Pclass', 'Sex', 'Age_bin', 'SibSp', 'Parch', 'Fare', 'Embarked','NameSr']]
X_test = test_df[['Pclass', 'Sex', 'Age_bin', 'SibSp', 'Parch', 'Fare', 'Embarked','NameSr']]

In [34]:
X_train.isnull().sum()

Pclass        0
Sex           0
Age_bin     177
SibSp         0
Parch         0
Fare          0
Embarked      0
NameSr        0
dtype: int64

In [28]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression


# Logistic Regression
logreg = LogisticRegression() #(C=0.1, penalty='l1', tol=1e-6)
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)

result_train = logreg.score(X_train, Y_train)
result_val = cross_val_score(logreg,X_train, Y_train, cv=5).mean()
print('taring score = %s , while validation score = %s' %(result_train , result_val))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [29]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('titanic.csv', index=False)
print('Exported')

NameError: name 'Y_pred' is not defined

In [30]:
# Random Forests
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [31]:
train_Y=train_df['Survived']
train_X=train_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked','NameSr']]
test_X = test_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked','NameSr']]

In [32]:
random_forest.fit(train_X, train_Y)
Y_pred_rf = random_forest.predict(test_X)
random_forest.score(train_X,train_Y)
acc_random_forest = round(random_forest.score(train_X, train_Y) * 100, 2)

print("Important features")
#pd.Series(random_forest.feature_importances_,train_X.columns).sort_values(ascending=True).plot.barh(width=0.8)
print('__'*30)
print(acc_random_forest)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [32]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred_rf})
submission.to_csv('titanic2.csv', index=False)
print('Exported')

Exported
